# Visualisations for Dashboard

## Imports

In [ ]:
from os import environ as ENV
from dotenv import load_dotenv

import pyodbc
import pandas as pd
import altair as alt
alt.data_transformers.enable("vegafusion")

DataTransformerRegistry.enable('vegafusion')

In [2]:
load_dotenv()
conn_str = (f"DRIVER={{{ENV['DB_DRIVER']}}};SERVER={ENV['DB_HOST']};"
            f"PORT={ENV['DB_PORT']};DATABASE={ENV['DB_NAME']};"
            f"UID={ENV['DB_USER']};PWD={ENV['DB_PASSWORD']};Encrypt=no;")

conn = pyodbc.connect(conn_str)

In [3]:
query = """
SELECT
    pr.*, p.*, c.*
FROM
    gamma.plant_reading pr
LEFT JOIN
    gamma.plant p
ON pr.plant_id = p.plant_id
LEFT JOIN
    gamma.city c
ON p.city_id = c.city_id;
"""
df_recordings = pd.read_sql(query, conn)
df_recordings.set_index("reading_id")

/var/folders/wc/2kss00k17yg_nb8945wzxzv40000gp/T/ipykernel_17898/3513501026.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_recordings = pd.read_sql(query, conn)


,plant_id,botanist_id,temperature,last_watered,soil_moisture,recording_taken,plant_id,plant_name,lat,lang,city_id,scientific_name,city_id,city_name,state_name
reading_id,,,,,,,,,,,,,,,
1,1,1,14,2025-09-25 13:51:41,32.97,2025-09-26 09:28:06.098,1,Venus flytrap,43.74,-11.51,1,None,1,Stammside,Albania
2,2,2,14,2025-09-25 14:58:34,34.15,2025-09-26 09:28:04.689,2,Corpse flower,47.84,-48.71,2,None,2,Floshire,American Samoa
3,3,3,80,2025-09-25 13:58:19,35.61,2025-09-26 09:28:05.186,3,Rafflesia arnoldii,-25.49,-36.13,3,None,3,Dale City,Mozambique
4,4,4,16,2025-09-25 14:56:07,36.58,2025-09-26 09:28:06.053,4,Black bat flower,63.37,46.40,4,None,4,West Tedboro,Taiwan
5,5,5,16,2025-09-25 13:57:08,29.81,2025-09-26 09:28:05.664,5,Pitcher plant,82.89,0.63,5,'Sarracenia catesbaei',5,North Felicia,Saint Kitts and Nevis
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5424,41,42,18,2025-09-25 13:08:03,26.13,2025-09-26 11:25:05.228,41,Medinilla Magnifica,-34.19,-78.70,42,'Medinilla magnifica',42,Kathrynville,Macao
5425,42,43,12,2025-09-25 14:50:57,-5.78,2025-09-26 11:25:05.402,42,Calliandra Haematocephala,83.92,-148.55,43,'Calliandra haematocephala',43,Gavinstad,Jersey
5426,43,44,15,2025-09-25 14:09:29,25.33,2025-09-26 11:25:04.943,43,Zamioculcas Zamiifolia,-82.18,-72.49,44,'Zamioculcas zamiifolia',44,Lake Norbertstead,Greenland


## Live Dashboard

### Temperature over Time

In [4]:
temp_recordings = pd.concat(
    [df_recordings["plant_name"], pd.to_datetime(df_recordings["recording_taken"]), df_recordings["temperature"]], axis=1)
temp_recordings = temp_recordings.rename(columns={"recording_taken": "time"})
temp_recordings["time"] = temp_recordings["time"].dt.floor('min')
temp_recordings

,plant_name,time,temperature
0,Venus flytrap,2025-09-26 09:28:00,14
1,Corpse flower,2025-09-26 09:28:00,14
2,Rafflesia arnoldii,2025-09-26 09:28:00,80
3,Black bat flower,2025-09-26 09:28:00,16
4,Pitcher plant,2025-09-26 09:28:00,16
...,...,...,...
5423,Medinilla Magnifica,2025-09-26 11:25:00,18
5424,Calliandra Haematocephala,2025-09-26 11:25:00,12
5425,Zamioculcas Zamiifolia,2025-09-26 11:25:00,15
5426,Crassula Ovata,2025-09-26 11:25:00,17


In [5]:
alt.Chart(temp_recordings, title="Temperature over Time").mark_line().encode(
    x="time:T",
    y="temperature:Q",
    color="plant_name:N"
)

alt.Chart(...)

### Soil Moisture Level over Time

In [6]:
soil_recordings = pd.concat(
    [df_recordings["plant_name"], pd.to_datetime(df_recordings["recording_taken"]), df_recordings["soil_moisture"]], axis=1)
soil_recordings = soil_recordings.rename(columns={"recording_taken": "time"})
soil_recordings["time"] = soil_recordings["time"].dt.floor('min')

soil_not_negative_filter = soil_recordings["soil_moisture"] >= 0
soil_recordings = soil_recordings[soil_not_negative_filter]
soil_recordings

,plant_name,time,soil_moisture
0,Venus flytrap,2025-09-26 09:28:00,32.97
1,Corpse flower,2025-09-26 09:28:00,34.15
2,Rafflesia arnoldii,2025-09-26 09:28:00,35.61
3,Black bat flower,2025-09-26 09:28:00,36.58
4,Pitcher plant,2025-09-26 09:28:00,29.81
...,...,...,...
5422,Begonia,2025-09-26 11:25:00,26.65
5423,Medinilla Magnifica,2025-09-26 11:25:00,26.13
5425,Zamioculcas Zamiifolia,2025-09-26 11:25:00,25.33
5426,Crassula Ovata,2025-09-26 11:25:00,26.40


In [7]:
alt.Chart(soil_recordings, title="Soil Moisture over Time").mark_line().encode(
    x="time:T",
    y="soil_moisture:Q",
    color="plant_name:N"
)

alt.Chart(...)

In [8]:
conn.close()